In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets sentencepiece

!pip uninstall -q -y transformers accelerate
!pip install -q transformers accelerate

!pip install -q evaluate

!pip install -q git+https://github.com/huggingface/accelerate

!git clone https://github.com/huggingface/transformers.git
!cd transformers
!pip install -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Cloning into 'transformers'...
remote: Enumerating objects: 203928, done.
remote: Counting objects: 100% (1753/1753), done.
remote: Compressing objects: 100% (711/711), done.
remote: Total 203928 (delta 1140),

In [ ]:
import json
import torch
from transformers import MBart50TokenizerFast, MBartForConditionalGeneration
from transformers import AutoTokenizer, MT5ForConditionalGeneration
# from parsivar import Normalizer
from datasets import Dataset
from tqdm.auto import tqdm
import collections
from transformers import AutoModelForQuestionAnswering
from transformers import Trainer
import evaluate
import os
import numpy as np
import random

In [ ]:
# with open('/content/drive/MyDrive/my_quran_dataset/sadeqi_test.json', 'r', encoding = 'utf-8-sig') as f:
with open('/content/drive/MyDrive/dev_v9.json', 'r', encoding = 'utf-8-sig') as f:
  dev_data_list = json.load(f)

with open('/content/drive/MyDrive/train_v9.json', 'r', encoding = 'utf-8-sig') as f:
  train_data_list = json.load(f)

total_data_list = dev_data_list + train_data_list
random.shuffle(total_data_list)

dev_number = len(total_data_list) * 0.1

dev_data = Dataset.from_list(total_data_list[:763])

dev_data_list = total_data_list[:763]

In [ ]:
with open('/content/drive/MyDrive/dev_10_percent2.json', 'w', encoding = 'utf-8-sig') as f:
  json.dump(dev_data_list, f)

In [ ]:
max_length = 384
stride = 128

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["pq_id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
metric = evaluate.load("squad")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# for each in os.listdir('/content/drive/MyDrive/my_quran_dataset'):
models_results = []
model_name_or_path = 'NegarSH/mbart'
tokenizer = MBart50TokenizerFast.from_pretrained(model_name_or_path)
model = MBartForConditionalGeneration.from_pretrained(model_name_or_path, use_safetensors=True).to(device)

predicted_output = []
ground_truth = []

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:
  ground_truth.append(qa['answers'][0]['text'].strip())

  question = qa['question']
  context = qa['context']


  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                max_length=64, output_scores = True, return_dict_in_generate = True)


  generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
  answer_scores = generated_answer["sequences_scores"]

  answers_dict = {}
  for ans, score in zip(generated_answers, answer_scores):
    if ans and ans != '[CLS]' and ans != '[SEP]' and ans in context and len(ans) <= 30 and len(ans)>0:
        answers_dict[ans] = score

  try:
    final_answer = max(answers_dict, key=answers_dict.get)
    final_score = answers_dict[final_answer]
  except:
    final_answer = ''
    final_score = 0
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer.strip(), "score": final_score})
  predicted_output.append(final_answer.strip())

  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  new_ex['answers']['text'] = [qa['answers'][0]['text']]
  theoretical_answers.append(new_ex)

models_results.append(predicted_answers)

# with open(dev_path, 'r', encoding = 'utf-8-sig') as f:
#   dev_data_list = json.load(f)

dev_data = Dataset.from_list(dev_data_list)

n_best = 20
max_answer_length = 30

checkpoint_list = ['NeginShams/albert-Quran_QA',
                  'NeginShams/xlm-roberta-Quran_QA',
                    'NeginShams/mbert-Quran_QA',
                    'NeginShams/parsbert-Quran_QA']

# def compute_metrics(start_logits, end_logits, features, examples):
for checkpoint in checkpoint_list:
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

    validation_dataset = dev_data.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dev_data.column_names,
    )

    reloaded_trainer = Trainer(
                    model = model,
                    tokenizer = tokenizer,
                    # other arguments if you have changed the defaults
                    )

    predictions, _, _ = reloaded_trainer.predict(validation_dataset)

    start_logits, end_logits = predictions

    features = validation_dataset

    examples = dev_data


    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["pq_id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"], 'score':best_answer["logit_score"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": "", 'score':0})

    models_results.append(predicted_answers)




model_name_or_path = 'NegarSH/mt5-Quran-QA'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = MT5ForConditionalGeneration.from_pretrained(model_name_or_path, use_safetensors=True).to(device)

predicted_output = []
ground_truth = []

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:
  ground_truth.append(qa['answers'][0]['text'].strip())

  question = qa['question']
  context = qa['context']


  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                max_length=64, output_scores = True, return_dict_in_generate = True)


  generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
  answer_scores = generated_answer["scores"]

  answers_dict = {}
  for ans, score in zip(generated_answers, answer_scores):
    if ans and ans != '[CLS]' and ans != '[SEP]' and ans in context and len(ans) <= 30 and len(ans)>0:
        answers_dict[ans] = score

  try:
    final_answer = max(answers_dict, key=answers_dict.get)
    final_score = answers_dict[final_answer]
  except:
    final_answer = ''
    final_score = 0
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer.strip(), "score": final_score})
  predicted_output.append(final_answer.strip())

  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  new_ex['answers']['text'] = [qa['answers'][0]['text']]
  theoretical_answers.append(new_ex)

models_results.append(predicted_answers)



sava_path = '/content/drive/MyDrive/original_data_predictions.pt'
torch.save(models_results, sava_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/763 [00:00<?, ? examples/s]

  0%|          | 0/763 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/763 [00:00<?, ? examples/s]

...

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
